In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

sns.set_theme()

# jupyter notebook full-width display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# no text wrapping
display(HTML("<style>.dataframe td { white-space: nowrap; }</style>"))

# pandas formatting
pd.set_option('display.float_format', '{:.3f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', 400)

In [2]:
df_SD = pd.read_pickle('df_SD.pickle')
df_FD = pd.read_pickle('df_FD.pickle')
df_LF = pd.read_pickle('df_LF.pickle')
df_LF_grouped = pd.read_pickle('df_LF_grouped.pickle')
df_Site = pd.read_pickle('df_Site.pickle')
df_TrapSupervisors = pd.read_pickle('df_TrapSupervisors.pickle')

# checking stuff

### why don't these match?
* answer: they do, but there is no matching sample to link them

In [20]:
df_LF[(df_LF.yy==2019) & (df_LF.mm==5) & (df_LF.dd==28) & (df_LF.site=="47")]

,yy,mm,dd,Time,river,week,site,loc,period,wt_lbs,wt_kg,lgth,freq,Flbin,DATETIME,SITE1,SITE2,SITE3,id,length_bin_id,FLAG_SITE,FLAG_AM_PM_PERIOD,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_NO_MATCHING_SAMPLE
11195,2019,5,28,NaN,MARGAREE,5,47,UPPER,PM,80.900,36.696,215,1,215,2019-05-28,47,<NA>,<NA>,2019052847,21.500,NaN,NaN,NaN,True
11196,2019,5,28,NaN,MARGAREE,5,47,UPPER,PM,80.900,36.696,220,6,220,2019-05-28,47,<NA>,<NA>,2019052847,22.000,NaN,NaN,NaN,True
11197,2019,5,28,NaN,MARGAREE,5,47,UPPER,PM,80.900,36.696,225,16,225,2019-05-28,47,<NA>,<NA>,2019052847,22.500,NaN,NaN,NaN,True
11198,2019,5,28,NaN,MARGAREE,5,47,UPPER,PM,80.900,36.696,230,26,230,2019-05-28,47,<NA>,<NA>,2019052847,23.000,NaN,NaN,NaN,True
11199,2019,5,28,NaN,MARGAREE,5,47,UPPER,PM,80.900,36.696,235,39,235,2019-05-28,47,<NA>,<NA>,2019052847,23.500,NaN,NaN,NaN,True
11200,2019,5,28,NaN,MARGAREE,5,47,UPPER,PM,80.900,36.696,240,51,240,2019-05-28,47,<NA>,<NA>,2019052847,24.000,NaN,NaN,NaN,True
11201,2019,5,28,NaN,MARGAREE,5,47,UPPER,PM,80.900,36.696,245,35,245,2019-05-28,47,<NA>,<NA>,2019052847,24.500,NaN,NaN,NaN,True
11202,2019,5,28,NaN,MARGAREE,5,47,UPPER,PM,80.900,36.696,250,27,250,2019-05-28,47,<NA>,<NA>,2019052847,25.000,NaN,NaN,NaN,True
11203,2019,5,28,NaN,MARGAREE,5,47,UPPER,PM,80.900,36.696,255,17,255,2019-05-28,47,<NA>,<NA>,2019052847,25.500,NaN,NaN,NaN,True
11204,2019,5,28,NaN,MARGAREE,5,47,UPPER,PM,80.900,36.696,260,5,260,2019-05-28,47,<NA>,<NA>,2019052847,26.000,NaN,NaN,NaN,True


In [22]:
df_SD[(df_SD.DATETIME.dt.year==2019) & (df_SD.DATETIME.dt.month==5) & (df_SD.DATETIME.dt.day==28)]

,DIST,RIVER,NAME,code,GEAR,SITE_NO,no_nets,YEAR,MM,DD,Week,catch_lbs,catch_kg,hours_fished,zone,last_name,comments,bycatch_sbass,bycatch_shad,bycatch_other,DATETIME,SITE1,SITE2,remarks,id,total_fish_preserved,total_fish_measured,AM_PM_PERIOD,wt_lbs,FLAG_DATETIME,FLAG_HOURS_FISHED,FLAG_SITE,FLAG_AM_PM_PERIOD_DISCREPANCIES
14959,2,SWMARG,Martin E Cameron,NaN,81,12,1,2019,5,28,5,350.000,158.800,5,lower,Cameron,NaN,NaN,NaN,NaN,2019-05-28,12,<NA>,NaN,2019052812,31.000,195.000,AM,71.900,NaN,NaN,NaN,NaN
14960,2,SWMARG,Robert Peters,NaN,81,25,1,2019,5,28,5,25.000,11.300,8,lower,Peters,NaN,NaN,NaN,NaN,2019-05-28,25,<NA>,NaN,2019052825,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14961,2,SWMARG,Pierre Chiasson,NaN,81,26,1,2019,5,28,5,200.000,90.700,7,lower,Chiasson,NaN,NaN,NaN,NaN,2019-05-28,26,<NA>,NaN,2019052826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14962,2,SWMARG,Gerard MacFarlane,NaN,81,28,1,2019,5,28,5,200.000,90.700,8,upper,MacFarlane,NaN,NaN,NaN,1 perch,2019-05-28,28,<NA>,bycatch_other: 1 perch,2019052828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14963,2,SWMARG,Daniel Stewart,NaN,81,33,1,2019,5,28,5,150.000,68.000,8,upper,Stewart,NaN,NaN,NaN,NaN,2019-05-28,33,<NA>,NaN,2019052833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14964,2,SWMARG,Stewart Gillis,NaN,81,41,1,2019,5,28,5,30.000,13.600,7,upper,Gillis,NaN,NaN,NaN,NaN,2019-05-28,41,<NA>,NaN,2019052841,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14965,2,SWMARG,Finley Stewart,NaN,81,44,NaN,2019,5,28,5,300.000,136.100,14,upper,Stewart,NaN,NaN,NaN,NaN,2019-05-28,44,<NA>,NaN,2019052844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14966,2,SWMARG,Mary E Gillis,NaN,81,49,1,2019,5,28,5,700.000,317.500,14,upper,Gillis,NaN,NaN,NaN,NaN,2019-05-28,49,<NA>,NaN,2019052849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14967,2,SWMARG,Bruce MacLellan,NaN,81,51,1,2019,5,28,5,15.000,6.800,6,upper,MacLellan,NaN,NaN,NaN,NaN,2019-05-28,51,<NA>,NaN,2019052851,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14968,2,SWMARG,Elizabeth MacKinnon,NaN,81,62,1,2019,5,28,5,500.000,226.800,10,upper,MacKinnon,NaN,NaN,NaN,NaN,2019-05-28,62,<NA>,NaN,2019052862,36.000,203.000,AM,74.100,NaN,NaN,NaN,NaN


In [42]:
# kevin followup question: how many ambiguous matches don't have a matching sample because there are multiple sample matches
ambiguous = set()
for i in list(df_SD[df_SD.id>2024000000].id):
    while i > 2024000000:
        i -= 1000000000
    ambiguous.add(i)
    
ambiguous = list(ambiguous)

In [90]:
for ambiguous_sample in ambiguous:
    YEAR, MONTH, DAY, SITE = (
        ambiguous_sample//1000000, 
        ambiguous_sample//10000 - 100*(ambiguous_sample//1000000), 
        ambiguous_sample//100 - 100*(ambiguous_sample//10000), 
        ambiguous_sample - 100*(ambiguous_sample//100)
    )
    print()
    print(YEAR, MONTH, DAY, SITE)
    # no exact matches
    display(df_LF[(df_LF.yy==YEAR) & (df_LF.mm==MONTH) & (df_LF.dd==DAY) & (df_LF.site==str(SITE))])
    display(df_FD[(df_FD.YEAR==YEAR) & (df_FD.MM==MONTH) & (df_FD.DD==DAY) & (df_FD.SITE==SITE)])


2004 6 10 1


,yy,mm,dd,Time,river,week,site,loc,period,wt_lbs,wt_kg,lgth,freq,Flbin,DATETIME,SITE1,SITE2,SITE3,id,length_bin_id,FLAG_SITE,FLAG_AM_PM_PERIOD,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_NO_MATCHING_SAMPLE


,YEAR,MM,DD,WEEK,SITE,PERIOD,CONDITION,FISH_NO,FL_WET,FL_FROZEN,FL_STD,WEIGHT,SPECIES,SEX,MATURITY,GONAD_WEIGHT,Ager_1,AGE_1,FSP_1,Comments_1,Ager_2,AGE_2,FSP_2,Comments_2,Ager_3,AGE_3,FSP_3,Comments_3,Envelop.Comments,DATETIME,AGE_notes_1,FSP_notes_1,AGE_notes_2,FSP_notes_2,AGE_notes_3,FSP_notes_3,SITE_notes,SITE1,SITE2,SITE3,fish_length,remarks,id,FLAG_SITE,FLAG_AM_PM_PERIOD,FLAG_SEX,FLAG_MATURITY,FLAG_FSP_1,FLAG_FL_STD,FLAG_FL_WET_FROZEN,FLAG_WEIGHT_OUTLIER,FLAG_GONAD_OUTLIER,FLAG_MULTIPLE_SAMPLE_POSSIBILITIES,FLAG_LEN_WT_RATIO_OUTLIER,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_NO_MATCHING_SAMPLE



1997 6 12 58


,yy,mm,dd,Time,river,week,site,loc,period,wt_lbs,wt_kg,lgth,freq,Flbin,DATETIME,SITE1,SITE2,SITE3,id,length_bin_id,FLAG_SITE,FLAG_AM_PM_PERIOD,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_NO_MATCHING_SAMPLE


...

,yy,mm,dd,Time,river,week,site,loc,period,wt_lbs,wt_kg,lgth,freq,Flbin,DATETIME,SITE1,SITE2,SITE3,id,length_bin_id,FLAG_SITE,FLAG_AM_PM_PERIOD,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_NO_MATCHING_SAMPLE


,YEAR,MM,DD,WEEK,SITE,PERIOD,CONDITION,FISH_NO,FL_WET,FL_FROZEN,FL_STD,WEIGHT,SPECIES,SEX,MATURITY,GONAD_WEIGHT,Ager_1,AGE_1,FSP_1,Comments_1,Ager_2,AGE_2,FSP_2,Comments_2,Ager_3,AGE_3,FSP_3,Comments_3,Envelop.Comments,DATETIME,AGE_notes_1,FSP_notes_1,AGE_notes_2,FSP_notes_2,AGE_notes_3,FSP_notes_3,SITE_notes,SITE1,SITE2,SITE3,fish_length,remarks,id,FLAG_SITE,FLAG_AM_PM_PERIOD,FLAG_SEX,FLAG_MATURITY,FLAG_FSP_1,FLAG_FL_STD,FLAG_FL_WET_FROZEN,FLAG_WEIGHT_OUTLIER,FLAG_GONAD_OUTLIER,FLAG_MULTIPLE_SAMPLE_POSSIBILITIES,FLAG_LEN_WT_RATIO_OUTLIER,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_NO_MATCHING_SAMPLE



1988 4 0 37


,yy,mm,dd,Time,river,week,site,loc,period,wt_lbs,wt_kg,lgth,freq,Flbin,DATETIME,SITE1,SITE2,SITE3,id,length_bin_id,FLAG_SITE,FLAG_AM_PM_PERIOD,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_NO_MATCHING_SAMPLE


,YEAR,MM,DD,WEEK,SITE,PERIOD,CONDITION,FISH_NO,FL_WET,FL_FROZEN,FL_STD,WEIGHT,SPECIES,SEX,MATURITY,GONAD_WEIGHT,Ager_1,AGE_1,FSP_1,Comments_1,Ager_2,AGE_2,FSP_2,Comments_2,Ager_3,AGE_3,FSP_3,Comments_3,Envelop.Comments,DATETIME,AGE_notes_1,FSP_notes_1,AGE_notes_2,FSP_notes_2,AGE_notes_3,FSP_notes_3,SITE_notes,SITE1,SITE2,SITE3,fish_length,remarks,id,FLAG_SITE,FLAG_AM_PM_PERIOD,FLAG_SEX,FLAG_MATURITY,FLAG_FSP_1,FLAG_FL_STD,FLAG_FL_WET_FROZEN,FLAG_WEIGHT_OUTLIER,FLAG_GONAD_OUTLIER,FLAG_MULTIPLE_SAMPLE_POSSIBILITIES,FLAG_LEN_WT_RATIO_OUTLIER,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_NO_MATCHING_SAMPLE


In [92]:
for ambiguous_sample in ambiguous:
    YEAR, MONTH, DAY, SITE = (
        ambiguous_sample//1000000, 
        ambiguous_sample//10000 - 100*(ambiguous_sample//1000000), 
        ambiguous_sample//100 - 100*(ambiguous_sample//10000), 
        ambiguous_sample - 100*(ambiguous_sample//100)
    )
    print()
    print(YEAR, MONTH, DAY, SITE)
    # partial matches
    display(df_LF[(df_LF.yy==YEAR) & (df_LF.mm==MONTH) & (df_LF.dd==DAY) & (df_LF.site.str.contains(str(SITE))==True)])
    display(df_FD[(df_FD.YEAR==YEAR) & (df_FD.MM==MONTH) & (df_FD.DD==DAY) & (df_FD.SITE.str.contains(str(SITE))==True)])


2004 6 10 1


,yy,mm,dd,Time,river,week,site,loc,period,wt_lbs,wt_kg,lgth,freq,Flbin,DATETIME,SITE1,SITE2,SITE3,id,length_bin_id,FLAG_SITE,FLAG_AM_PM_PERIOD,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_NO_MATCHING_SAMPLE


,YEAR,MM,DD,WEEK,SITE,PERIOD,CONDITION,FISH_NO,FL_WET,FL_FROZEN,FL_STD,WEIGHT,SPECIES,SEX,MATURITY,GONAD_WEIGHT,Ager_1,AGE_1,FSP_1,Comments_1,Ager_2,AGE_2,FSP_2,Comments_2,Ager_3,AGE_3,FSP_3,Comments_3,Envelop.Comments,DATETIME,AGE_notes_1,FSP_notes_1,AGE_notes_2,FSP_notes_2,AGE_notes_3,FSP_notes_3,SITE_notes,SITE1,SITE2,SITE3,fish_length,remarks,id,FLAG_SITE,FLAG_AM_PM_PERIOD,FLAG_SEX,FLAG_MATURITY,FLAG_FSP_1,FLAG_FL_STD,FLAG_FL_WET_FROZEN,FLAG_WEIGHT_OUTLIER,FLAG_GONAD_OUTLIER,FLAG_MULTIPLE_SAMPLE_POSSIBILITIES,FLAG_LEN_WT_RATIO_OUTLIER,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_NO_MATCHING_SAMPLE



1997 6 12 58


,yy,mm,dd,Time,river,week,site,loc,period,wt_lbs,wt_kg,lgth,freq,Flbin,DATETIME,SITE1,SITE2,SITE3,id,length_bin_id,FLAG_SITE,FLAG_AM_PM_PERIOD,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_NO_MATCHING_SAMPLE


...

,yy,mm,dd,Time,river,week,site,loc,period,wt_lbs,wt_kg,lgth,freq,Flbin,DATETIME,SITE1,SITE2,SITE3,id,length_bin_id,FLAG_SITE,FLAG_AM_PM_PERIOD,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_NO_MATCHING_SAMPLE


,YEAR,MM,DD,WEEK,SITE,PERIOD,CONDITION,FISH_NO,FL_WET,FL_FROZEN,FL_STD,WEIGHT,SPECIES,SEX,MATURITY,GONAD_WEIGHT,Ager_1,AGE_1,FSP_1,Comments_1,Ager_2,AGE_2,FSP_2,Comments_2,Ager_3,AGE_3,FSP_3,Comments_3,Envelop.Comments,DATETIME,AGE_notes_1,FSP_notes_1,AGE_notes_2,FSP_notes_2,AGE_notes_3,FSP_notes_3,SITE_notes,SITE1,SITE2,SITE3,fish_length,remarks,id,FLAG_SITE,FLAG_AM_PM_PERIOD,FLAG_SEX,FLAG_MATURITY,FLAG_FSP_1,FLAG_FL_STD,FLAG_FL_WET_FROZEN,FLAG_WEIGHT_OUTLIER,FLAG_GONAD_OUTLIER,FLAG_MULTIPLE_SAMPLE_POSSIBILITIES,FLAG_LEN_WT_RATIO_OUTLIER,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_NO_MATCHING_SAMPLE



1988 4 0 37


,yy,mm,dd,Time,river,week,site,loc,period,wt_lbs,wt_kg,lgth,freq,Flbin,DATETIME,SITE1,SITE2,SITE3,id,length_bin_id,FLAG_SITE,FLAG_AM_PM_PERIOD,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_NO_MATCHING_SAMPLE


,YEAR,MM,DD,WEEK,SITE,PERIOD,CONDITION,FISH_NO,FL_WET,FL_FROZEN,FL_STD,WEIGHT,SPECIES,SEX,MATURITY,GONAD_WEIGHT,Ager_1,AGE_1,FSP_1,Comments_1,Ager_2,AGE_2,FSP_2,Comments_2,Ager_3,AGE_3,FSP_3,Comments_3,Envelop.Comments,DATETIME,AGE_notes_1,FSP_notes_1,AGE_notes_2,FSP_notes_2,AGE_notes_3,FSP_notes_3,SITE_notes,SITE1,SITE2,SITE3,fish_length,remarks,id,FLAG_SITE,FLAG_AM_PM_PERIOD,FLAG_SEX,FLAG_MATURITY,FLAG_FSP_1,FLAG_FL_STD,FLAG_FL_WET_FROZEN,FLAG_WEIGHT_OUTLIER,FLAG_GONAD_OUTLIER,FLAG_MULTIPLE_SAMPLE_POSSIBILITIES,FLAG_LEN_WT_RATIO_OUTLIER,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_NO_MATCHING_SAMPLE


In [93]:
# maybe there the site number was written down wrong, some partial matches (excluding site)
for ambiguous_sample in ambiguous:
    YEAR, MONTH, DAY, SITE = (
        ambiguous_sample//1000000, 
        ambiguous_sample//10000 - 100*(ambiguous_sample//1000000), 
        ambiguous_sample//100 - 100*(ambiguous_sample//10000), 
        ambiguous_sample - 100*(ambiguous_sample//100)
    )
    print()
    print(YEAR, MONTH, DAY, SITE)
    # very partial matches
    display(df_LF[(df_LF.yy==YEAR) & (df_LF.mm==MONTH) & (df_LF.dd==DAY)].head())
    display(df_FD[(df_FD.YEAR==YEAR) & (df_FD.MM==MONTH) & (df_FD.DD==DAY)].head())


2004 6 10 1


,yy,mm,dd,Time,river,week,site,loc,period,wt_lbs,wt_kg,lgth,freq,Flbin,DATETIME,SITE1,SITE2,SITE3,id,length_bin_id,FLAG_SITE,FLAG_AM_PM_PERIOD,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_NO_MATCHING_SAMPLE


,YEAR,MM,DD,WEEK,SITE,PERIOD,CONDITION,FISH_NO,FL_WET,FL_FROZEN,FL_STD,WEIGHT,SPECIES,SEX,MATURITY,GONAD_WEIGHT,Ager_1,AGE_1,FSP_1,Comments_1,Ager_2,AGE_2,FSP_2,Comments_2,Ager_3,AGE_3,FSP_3,Comments_3,Envelop.Comments,DATETIME,AGE_notes_1,FSP_notes_1,AGE_notes_2,FSP_notes_2,AGE_notes_3,FSP_notes_3,SITE_notes,SITE1,SITE2,SITE3,fish_length,remarks,id,FLAG_SITE,FLAG_AM_PM_PERIOD,FLAG_SEX,FLAG_MATURITY,FLAG_FSP_1,FLAG_FL_STD,FLAG_FL_WET_FROZEN,FLAG_WEIGHT_OUTLIER,FLAG_GONAD_OUTLIER,FLAG_MULTIPLE_SAMPLE_POSSIBILITIES,FLAG_LEN_WT_RATIO_OUTLIER,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_NO_MATCHING_SAMPLE



1997 6 12 58


,yy,mm,dd,Time,river,week,site,loc,period,wt_lbs,wt_kg,lgth,freq,Flbin,DATETIME,SITE1,SITE2,SITE3,id,length_bin_id,FLAG_SITE,FLAG_AM_PM_PERIOD,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_NO_MATCHING_SAMPLE
3024,1997,6,12,NaN,SW MARGAREE,7,26,LOWER,AM,118.000,53.500,220,3,220,1997-06-12,26,<NA>,<NA>,1997061226,22.000,NaN,NaN,NaN,NaN
3025,1997,6,12,NaN,SW MARGAREE,7,26,LOWER,AM,118.000,53.500,225,4,225,1997-06-12,26,<NA>,<NA>,1997061226,22.500,NaN,NaN,NaN,NaN
3026,1997,6,12,NaN,SW MARGAREE,7,26,LOWER,AM,118.000,53.500,230,19,230,1997-06-12,26,<NA>,<NA>,1997061226,23.000,NaN,NaN,NaN,NaN
3027,1997,6,12,NaN,SW MARGAREE,7,26,LOWER,AM,118.000,53.500,235,52,235,1997-06-12,26,<NA>,<NA>,1997061226,23.500,NaN,NaN,NaN,NaN
3028,1997,6,12,NaN,SW MARGAREE,7,26,LOWER,AM,118.000,53.500,240,70,240,1997-06-12,26,<NA>,<NA>,1997061226,24.000,NaN,NaN,NaN,NaN


...

,yy,mm,dd,Time,river,week,site,loc,period,wt_lbs,wt_kg,lgth,freq,Flbin,DATETIME,SITE1,SITE2,SITE3,id,length_bin_id,FLAG_SITE,FLAG_AM_PM_PERIOD,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_NO_MATCHING_SAMPLE


,YEAR,MM,DD,WEEK,SITE,PERIOD,CONDITION,FISH_NO,FL_WET,FL_FROZEN,FL_STD,WEIGHT,SPECIES,SEX,MATURITY,GONAD_WEIGHT,Ager_1,AGE_1,FSP_1,Comments_1,Ager_2,AGE_2,FSP_2,Comments_2,Ager_3,AGE_3,FSP_3,Comments_3,Envelop.Comments,DATETIME,AGE_notes_1,FSP_notes_1,AGE_notes_2,FSP_notes_2,AGE_notes_3,FSP_notes_3,SITE_notes,SITE1,SITE2,SITE3,fish_length,remarks,id,FLAG_SITE,FLAG_AM_PM_PERIOD,FLAG_SEX,FLAG_MATURITY,FLAG_FSP_1,FLAG_FL_STD,FLAG_FL_WET_FROZEN,FLAG_WEIGHT_OUTLIER,FLAG_GONAD_OUTLIER,FLAG_MULTIPLE_SAMPLE_POSSIBILITIES,FLAG_LEN_WT_RATIO_OUTLIER,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_NO_MATCHING_SAMPLE
4503,1988,5,23,<NA>,12,NaN,Fresh,1,253,<NA>,253.000,208.000,A,M,4,NaN,NaN,4,4,NaN,NaN,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,NaN,1988-05-23,4,4,NaN,NaN,NaN,NaN,12,12,<NA>,<NA>,253,SITE_notes: 12; AGE_notes_1: 4; FSP_notes_1: 4,1988052312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4504,1988,5,23,<NA>,12,NaN,Fresh,2,258,<NA>,258.000,236.000,A,M,4,NaN,NaN,4,4,NaN,NaN,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,NaN,1988-05-23,4,4,NaN,NaN,NaN,NaN,12,12,<NA>,<NA>,258,SITE_notes: 12; AGE_notes_1: 4; FSP_notes_1: 4,1988052312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4505,1988,5,23,<NA>,12,NaN,Fresh,3,245,<NA>,245.000,NaN,A,M,4,NaN,NaN,4,4,NaN,NaN,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,NaN,1988-05-23,4,4,NaN,NaN,NaN,NaN,12,12,<NA>,<NA>,245,SITE_notes: 12; AGE_notes_1: 4; FSP_notes_1: 4,1988052312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4506,1988,5,23,<NA>,12,NaN,Fresh,4,250,<NA>,250.000,234.000,A,M,4,NaN,NaN,4,4,NaN,NaN,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,NaN,1988-05-23,4,4,NaN,NaN,NaN,NaN,12,12,<NA>,<NA>,250,SITE_notes: 12; AGE_notes_1: 4; FSP_notes_1: 4,1988052312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4507,1988,5,23,<NA>,12,NaN,Fresh,5,245,<NA>,245.000,209.000,A,F,4,29.100,NaN,3,3,NaN,NaN,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,NaN,1988-05-23,3,3,NaN,NaN,NaN,NaN,12,12,<NA>,<NA>,245,SITE_notes: 12; AGE_notes_1: 3; FSP_notes_1: 3,1988052312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



1988 4 0 37


,yy,mm,dd,Time,river,week,site,loc,period,wt_lbs,wt_kg,lgth,freq,Flbin,DATETIME,SITE1,SITE2,SITE3,id,length_bin_id,FLAG_SITE,FLAG_AM_PM_PERIOD,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_NO_MATCHING_SAMPLE


,YEAR,MM,DD,WEEK,SITE,PERIOD,CONDITION,FISH_NO,FL_WET,FL_FROZEN,FL_STD,WEIGHT,SPECIES,SEX,MATURITY,GONAD_WEIGHT,Ager_1,AGE_1,FSP_1,Comments_1,Ager_2,AGE_2,FSP_2,Comments_2,Ager_3,AGE_3,FSP_3,Comments_3,Envelop.Comments,DATETIME,AGE_notes_1,FSP_notes_1,AGE_notes_2,FSP_notes_2,AGE_notes_3,FSP_notes_3,SITE_notes,SITE1,SITE2,SITE3,fish_length,remarks,id,FLAG_SITE,FLAG_AM_PM_PERIOD,FLAG_SEX,FLAG_MATURITY,FLAG_FSP_1,FLAG_FL_STD,FLAG_FL_WET_FROZEN,FLAG_WEIGHT_OUTLIER,FLAG_GONAD_OUTLIER,FLAG_MULTIPLE_SAMPLE_POSSIBILITIES,FLAG_LEN_WT_RATIO_OUTLIER,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_NO_MATCHING_SAMPLE


# combine Age 2 and Age 3 columns

In [9]:
# check if age2 and age3 occur at the same time
(
    df_FD[df_FD['Ager_2'].notnull() & df_FD['Ager_3'].notnull()].shape[0],
    df_FD[df_FD['AGE_2'].notnull() & df_FD['AGE_3'].notnull()].shape[0],
    df_FD[df_FD['FSP_2'].notnull() & df_FD['FSP_3'].notnull()].shape[0], 
    df_FD[df_FD['Comments_2'].notnull() & df_FD['Comments_3'].notnull()].shape[0]
)

(0, 0, 0, 0)

In [12]:
# how many Ager 3 columns
df_FD[df_FD.Ager_3.notnull() | df_FD.AGE_3.notnull() | df_FD.FSP_3.notnull() | df_FD.Comments_3.notnull()].shape[0]

2339

In [10]:
# combine aging 2 and aging 3 into one set of data
df_FD['Ager_2'] = df_FD['Ager_2'].fillna(df_FD['Ager_3'])
df_FD['AGE_2'] = df_FD['AGE_2'].fillna(df_FD['AGE_3'])
df_FD['FSP_2'] = df_FD['FSP_2'].fillna(df_FD['FSP_3'])
df_FD['Comments_2'] = df_FD['Comments_2'].fillna(df_FD['Comments_3'])

# more changes

In [13]:
...

Ellipsis

# Save Data

In [ ]:
# save pickle files of dataframes
if save_pickles := False:
    df_SD.to_pickle('df_SD.pickle')
    df_FD.to_pickle('df_FD.pickle')
    df_LF.to_pickle('df_LF.pickle')
    df_LF_grouped.to_pickle('df_LF_grouped.pickle')
    df_Site.to_pickle('df_Site.pickle')
    df_TrapSupervisors.to_pickle('df_TrapSupervisors.pickle')

# save files to this workbook drive
if save_csv := False:
    df_SD.to_csv('gaspereau_sample_data.csv', index=False)
    df_FD.to_csv('gaspereau_fish_details.csv', index=False)
    df_LF.to_csv('gaspereau_length_frequencies.csv', index=False)
    df_Site.to_csv('gaspereau_sites.csv', index=False)
    df_TrapSupervisors.to_csv('gaspereau_trap_supervisors.csv', index=False)
    df_LF_grouped.to_csv('gaspereau_LF_grouped.csv', index=False)
    
# upload to temp folder for import into Kevin's local dm_apps
import_file_location = 'C:\\Users\\CARRK\\Documents\\Repositories\\dm_app_root\\dm_apps\\herring\\temp\\'

if upload_csv_to_temp_folder := False:
    df_SD.to_csv(import_file_location + 'gaspereau_sample_data.csv', index=False)
    df_FD.to_csv(import_file_location + 'gaspereau_fish_details.csv', index=False)
    df_Site.to_csv(import_file_location + 'gaspereau_sites.csv', index=False)
    df_TrapSupervisors.to_csv(import_file_location + 'gaspereau_trap_supervisors.csv', index=False)
    df_LF_grouped.to_csv(import_file_location + 'gaspereau_LF_grouped.csv', index=False)